# Passenger Satisfaction Classification
This notebook uses pandas and scikit-learn to train a classifier for airline passenger satisfaction.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

## Load the data

In [ ]:
train_df = pd.read_csv('train.csv').iloc[:, 1:]
train_df = train_df.drop(columns=['id'])


### Inspect missing values

In [ ]:
train_df.isna().sum()

In [ ]:
numeric_cols = train_df.select_dtypes(include='number').columns
for col in numeric_cols:
    if train_df[col].isna().any():
        median = train_df[col].median()
        train_df[col] = train_df[col].fillna(median)

Missing numeric values are replaced with the column median.

## Descriptive statistics

In [ ]:
print('Average age:', train_df['Age'].mean())
print('Satisfaction counts:')
print(train_df['satisfaction'].value_counts())
train_df.describe()

## Train/test split

In [ ]:
X = train_df.drop('satisfaction', axis=1)
y = train_df['satisfaction']

# Columns from the original script
numeric_features = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']

categorical_features = ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## Build the preprocessing and modeling pipeline

In [ ]:
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

clf = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(max_iter=1000))
])

## Train the model

In [ ]:
clf.fit(X_train, y_train)

## Evaluate

In [ ]:
y_pred = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))